In [4]:
#%pip install beautifulsoup4

In [1]:
import requests
from bs4 import BeautifulSoup as bs
import pandas as pd
import json
import stardog
import os
import io
from langchain_community.graphs import Neo4jGraph

- Buscar os links para a tese no Neo4j
- Buscar o link do PDF no repositório da universidade
- Baixar o PDF
- Extrair o texto do PDF 

Buscar links para a tese no Neo4j

Conectando ao Neo4j 

neo4j://localhost:7687

In [4]:

# Neo4j variables
NEO4J_URL = os.getenv("NEO4J_URL")
NEO4J_USERNAME =os.getenv("NEO4J_USERNAME")
NEO4J_PASSWORD = os.getenv("NEO4J_PASSWORD")

#Connecting to the graph
graph = Neo4jGraph(
    url=NEO4J_URL,
    username=NEO4J_USERNAME,
    password=NEO4J_PASSWORD
)

In [ ]:
# Stardog variables
#STARDOG_ENDPOINT = os.getenv('STARDOG_ENDPOINT')
#STARDOG_USERNAME = os.getenv("STARDOG_USERNAME")
#STARDOG_PASSWORD = os.getenv("STARDOG_PASSWORD")

#connection_details = {
#    'endpoint': STARDOG_ENDPOINT,
#    'username': STARDOG_USERNAME,
#    'password': STARDOG_PASSWORD
#}

In [ ]:
#database_name = 'IndigenousSlavery'
#conn = stardog.Connection(database_name, **connection_details)

In [11]:
#Saprql query
query1 = """
SELECT ?thesis ?link
WHERE {
  ?thesis <http://purl.org/dc/terms/publisher> <tag:stardog:api:USP>.
  ?thesis a <http://purl.org/ontology/bibo/Thesis>.
  ?thesis <http://purl.org/ontology/bibo/repository> ?link.  
}
"""

#csv_results = conn.select(query1, content_type='text/csv')
#thesis_abstract = pd.read_csv(io.BytesIO(csv_results))
#thesis_abstract


#Cypher query 
query2 = """
MATCH  (t:Thesis)-[:publisher]-(uni:University{uri:'tag:stardog:api:USP'})
RETURN  t["uri"] as Thesis, t["repository"] as repository 
"""

results = graph.query(query2)
# Convert the results to a DataFrame
thesis_repository = pd.DataFrame(results, columns=["Thesis", "repository"])
thesis_repository

#2213 / 2208

,Thesis,repository
0,tag:stardog:api:substituicao_do_negro_de_fumo_...,https://www.teses.usp.br/teses/disponiveis/3/3...
1,tag:stardog:api:diversidade_na_unidade_a_tradi...,https://www.teses.usp.br/teses/disponiveis/71/...
2,tag:stardog:api:kuxima_paa_dizque_antigamente_...,https://www.teses.usp.br/teses/disponiveis/8/8...
3,tag:stardog:api:vai_da_muito_trabalho_cultura_...,https://www.teses.usp.br/teses/disponiveis/48/...
4,tag:stardog:api:access_to_maternal_reproductiv...,https://www.teses.usp.br/teses/disponiveis/22/...
...,...,...
2203,tag:stardog:api:entre_o_tipo_e_o_sujeito_os_re...,http://www.teses.usp.br/teses/disponiveis/27/2...
2204,tag:stardog:api:a_equacao_de_blackscholes_com_...,http://www.teses.usp.br/teses/disponiveis/55/5...
2205,tag:stardog:api:o_feitor_ausente_estudo_sobre_...,https://teses.usp.br/teses/disponiveis/8/8138/...
2206,tag:stardog:api:um_ator_de_fronteira_uma_anali...,http://www.teses.usp.br/teses/disponiveis/27/2...


In [13]:
url = thesis_repository['repository'][0]
url

'https://www.teses.usp.br/teses/disponiveis/3/3133/tde-10082023-152417/'

In [ ]:
#import certifi
#import ssl
#import urllib3

In [14]:
#Fazer requisição e parsear o arquivo html
f = requests.get(url, verify=False).text#, verify=False) 
soup = bs(f, "html.parser")

c:\Users\facordei\OneDrive - Capgemini\Documents\GitHub\Indigenous-Slavery-KG\.venv-IndigSlave\Lib\site-packages\urllib3\connectionpool.py:1064: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.teses.usp.br'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
c:\Users\facordei\OneDrive - Capgemini\Documents\GitHub\Indigenous-Slavery-KG\.venv-IndigSlave\Lib\site-packages\urllib3\connectionpool.py:1064: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.teses.usp.br'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


In [16]:
#Coletando link para as teses
for doc in soup.find_all('a', href=True):
    if doc['href'].endswith('.pdf'):
        path = doc['href']
        prefix_uni = 'https://www.teses.usp.br'
        link = prefix_uni + path

link

'https://www.teses.usp.br/teses/disponiveis/3/3133/tde-10082023-152417/publico/FabioJoseEsperCorr.pdf'

In [58]:
#fazer download do arquivo
def download_file(url, local_filename):
    # NOTE the stream=True parameter below
    r = requests.get(url, verify=False, stream=True)
    # raise_for_status() is not needed with stream=True
    # r.raise_for_status()
    with open(local_filename, 'wb') as f:
        for chunk in r.iter_content(chunk_size=8192):
            f.write(chunk)
    return local_filename

In [63]:
download_file(link, 'garavazo.pdf')

c:\Users\facordei\OneDrive - Capgemini\Documents\GitHub\Indigenous-Slavery-KG\.venv-IndigSlave\Lib\site-packages\urllib3\connectionpool.py:1064: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.teses.usp.br'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


'garavazo.pdf'

In [1]:
#!python -m spacy download pt_core_news_lg
#!python -m spacy download en_core_web_lg
#%pip install -U scikit-learn

In [1]:
import os
import io
import stardog
import pandas as pd
import spacy
from spacy import displacy
from sklearn.cluster import DBSCAN
import numpy as np
import requests
from unidecode import unidecode
import re
import json
from langdetect import detect

### Specify Stardog connection details